In [1]:
# Importing required libraries 
import os
import pickle
import pandas as pd
import numpy as np

import altair as alt

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

def predict_demand_for_region(Precipitation, RelativeHumidity, AirTemperature, WetBulbTemperature, DewTemperature,
                              SeaPressure, StationPressure, Month, Day, DayOfYear, Hour,
                              Season_Autumn, Season_Spring, Season_Summer, Season_Winter,
                              TimeOfDay_Afternoon, TimeOfDay_Evening, TimeOfDay_Morning, TimeOfDay_Night,
                              IsWeekend_False, IsWeekend_True, model):
    predicted_demand_next_24_hours = []
    for hour in range(24):
        # Prepare the input features for prediction
        input_features = [[Precipitation, RelativeHumidity, AirTemperature, WetBulbTemperature, DewTemperature,
                           SeaPressure, StationPressure, Month, Day, Hour + hour, DayOfYear,
                           Season_Autumn, Season_Spring, Season_Summer, Season_Winter,
                           TimeOfDay_Afternoon, TimeOfDay_Evening, TimeOfDay_Morning, TimeOfDay_Night,
                           IsWeekend_False, IsWeekend_True]]
        
        # Make predictions using the provided model
        predicted_demand = model.predict(input_features)[0]
        
        predicted_demand_next_24_hours.append(predicted_demand)
    
    # Create a list of hour values from 1 to 24
    hours = list(range(1, 25))
    
    # Create a DataFrame with 'Hour' as the index and 'Predicted Demand' as the column
    predicted_demand_df = pd.DataFrame({'Hour': hours, 'Predicted Demand': predicted_demand_next_24_hours})
    # Calculate cumulative sum of predicted demands
    predicted_demand_df['Cumulative Demand'] = predicted_demand_df['Predicted Demand'].cumsum()
    # Calculate average hourly predicted demand
    predicted_demand_df['Average Hourly Demand'] = predicted_demand_df['Cumulative Demand'] / (predicted_demand_df['Hour'])
    
    return predicted_demand_df

def create_interactive_demand_plot(predicted_demand_df, region_name):
    
    # Define the interactive area plot with customized axis properties and marks for each hour
    area_plot = alt.Chart(predicted_demand_df).mark_area(color='lightblue', opacity=0.5, line={'color': 'darkblue'}).encode(
        x=alt.X('Hour', axis=alt.Axis(title='Hour', grid=True, gridColor='gray', gridWidth=0.5),
                scale=alt.Scale(domain=(0, 24))), 
        y=alt.Y('Predicted Demand',
                axis=alt.Axis(title='Predicted Demand (MW)', grid=True, gridColor='gray', gridWidth=0.5),
                scale=alt.Scale(domain=(0, 11000))),  
        tooltip=[
            'Hour',
            alt.Tooltip('Predicted Demand', title='Predicted Demand (MW)', format='.3f'),
            alt.Tooltip('Cumulative Demand', title='Predicted Total Demand (MW)', format='.3f'),
            alt.Tooltip('Average Hourly Demand', title='Predicted Average Demand per Hour (MW)', format='.3f')
        ]
    ).properties(
        width=800,
        height=400,
        title=f'Predicted Demand Over 24 Hours for {region_name}'
    ).interactive() 

    # Add marks for each hour
    hour_marks = area_plot.mark_circle(color='black').encode(
        x='Hour',
        y='Predicted Demand',
        tooltip=[
            'Hour',
            alt.Tooltip('Predicted Demand', title='Predicted Demand (MW)', format='.3f'),
            alt.Tooltip('Cumulative Demand', title='Total Predicted Demand (MW)', format='.3f'),
            alt.Tooltip('Average Hourly Demand', title='Average Hourly Demand (MW)', format='.3f')
        ]
    )

    # Combine the area plot and hour marks
    combined_plot = area_plot + hour_marks
    
    return combined_plot